## Trip Data Processing

### Purpose

* Create datasets for Tableau visualization from raw trip data files.

### Rationale

* Monthly raw trip data files are quite large, and two files are needed for year-over-year comparisons.  Having Tableau use the files directly will slow down performance.  Pre-processed summary files will still provide useful data to Tableau, but in a much more compact format.

### Requirements

* Pandas (0.24.2)
* Numpy (0.16.4)

### Input / Output

* Input files named according to `{yyyy}{mm}-citibike-tripdata.csv` where yyyy is the current year, and mm is the month of interest.  The system will look for two files, one with year yyyy, and another with year yyyy - 1.

* Output file named `{yyyy}{mm}-citibike-hourly-summary.csv`

### <span style="color:blue">Required User Input</span>

In [4]:
valid = False
attempts = 0  # Tracked to avoid infinite loops
while (not valid) and (attempts < 6):
    attempts += 1
    if attempts == 6:
        print('Too many invalid attempts.  Exiting loop.  *This notebook will not process data*.')
        break
    try:
        mmyyyy = input('Please enter the two-digit month and four-digit year for analysis (e.g. 05-2019)')
        mm = int(mmyyyy[0:2])
        yyyy = int(mmyyyy[-4:])
        print('Thank you.')
        valid = True
    except ValueError:
        print('Invalid entry.  Pleae try again.')

Please enter the two-digit month and four-digit year for analysis (e.g. 05-2019)go to hell
Invalid entry.  Pleae try again.
Please enter the two-digit month and four-digit year for analysis (e.g. 05-2019)05-2019


### Set-Up / Import 

In [18]:
import pandas as pd
import numpy as np
import matplotlib as mpl
%matplotlib inline

In [36]:
# Compose import filenames
trip_data_dir = '../data/trips/'
if mm < 10:
    month_str = '0' + str(mm)
else:
    month_str = str(mm)
this_year_str = str(yyyy)
last_year_str = str(yyyy - 1)
this_year_filename = trip_data_dir + this_year_str + month_str + '-citibike-tripdata.csv'
last_year_filename = trip_data_dir + last_year_str + month_str + '-citibike-tripdata.csv'
# Get data
this_yr_trips = pd.read_csv(this_year_filename, parse_dates = ['starttime','stoptime'])
last_yr_trips = pd.read_csv(last_year_filename, parse_dates = ['starttime', 'stoptime'])

### Basic Data Checking

In [37]:
# Examine data
this_yr_trips.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.901,2019-05-01 00:02:21.517,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.021,2019-05-01 00:12:37.692,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.627,2019-05-01 00:38:33.171,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.334,2019-05-01 00:02:42.520,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.184,2019-05-01 00:02:40.648,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


In [38]:
last_yr_trips.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,367,2018-05-01 05:06:16.584,2018-05-01 05:12:23.965,72,W 52 St & 11 Ave,40.767272,-73.993929,514,12 Ave & W 40 St,40.760875,-74.002777,30567,Subscriber,1965,1
1,1313,2018-05-01 06:25:49.425,2018-05-01 06:47:42.712,72,W 52 St & 11 Ave,40.767272,-73.993929,426,West St & Chambers St,40.717548,-74.013221,18965,Subscriber,1956,1
2,1798,2018-05-01 06:40:26.445,2018-05-01 07:10:25.179,72,W 52 St & 11 Ave,40.767272,-73.993929,3435,Grand St & Elizabeth St,40.718822,-73.995960,30241,Subscriber,1959,2
3,518,2018-05-01 07:06:02.973,2018-05-01 07:14:41.004,72,W 52 St & 11 Ave,40.767272,-73.993929,477,W 41 St & 8 Ave,40.756405,-73.990026,28985,Subscriber,1986,1
4,109,2018-05-01 07:26:32.345,2018-05-01 07:28:21.542,72,W 52 St & 11 Ave,40.767272,-73.993929,530,11 Ave & W 59 St,40.771522,-73.990541,14556,Subscriber,1991,1


In [39]:
# Check row counts
print(str(len(this_yr_trips)) + ' rows for ' + this_year_str)
print(str(len(last_yr_trips)) + ' rows for ' + last_year_str)

1924563 rows for 2019
1824710 rows for 2018


In [40]:
# Check that columns are the same 
this_yr_trips.columns == last_yr_trips.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [41]:
this_yr_trips.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

### Generate Hourly Station Data

In [ ]:
def frac_subscribers(series):
    """Returns that fraction of DataFrame Series entries that are 'Subscriber', ignores na"""
    num_subscribers = len(list(filter(lambda x: x == 'Subscriber', series)))
    series_count = series.count()
    return num_subscribers / series_count

In [29]:
def frac_in_age_range(series, current_year = yyyy, lower_limit = 0, upper_limit = 120):
    """Returns fraction of data series that represents people between lower_limit (inclusive) 
    and upper_limit (exclusive) of age, given a DataFrame Series with birth years and the 
    current year.  Note that ages are not exact, but equal the difference in current_year and the
    birth year.  Limits are converted to integers."""
    try:
        numeric_series = series.astype('int').dropna()
        lower_limit = int(lower_limit)
        upper_limit = int(upper_limit)
        num_in_range = len(list(filter(lambda x: ((current_year - x) >= lower_limit) and
                                                ((current_year - x) < upper_limit), numeric_series)))
        series_count = len(numeric_series)
        return num_in_range / series_count
    except ValueError:
        return 0

In [ ]:
def frac_male(series)
    """Returns fraction of data series that represents males, given a DataFrame series encoded
    as 0 = unknown, 1 = male, 2 = female"""
    num_male = len(list(filter(lambda x: x == 1, series)))
    num_female = len(list(filter(lambda x: x == 2, series)))
    return num_male / (num_male + num_female)

In [31]:
# Build aggregation dictionary for use with groupby
#  -- tripduration is very unevenly distributed (lots of long-length outliers, so use median
# -- use custom functions for usertype, gender, and birth year 
# -- Note that yyyy should already be set to the current year
agg_dict = {'tripduration':['median','count'], 'start station latitude':'first', 
            'start station longitude':'first','usertype':'frac_subscribers',
            'gender':'frac_male','birth year' : [lambda x : frac_in_age_range(x, yyyy, 0, 25),
                                                lambda x : frac_in_age_range(x, yyyy, 25, 35),
                                                lambda x : frac_in_age_range(x, yyyy, 35, 45),
                                                lambda x : frac_in_age_range(x, yyyy, 45, 55),
                                                lambda x : frac_in_age_range(x, yyyy, 55, 199)]}

In [42]:
this_yr_trips.starttime.dt.date.value_counts()

2019-05-31    83347
2019-05-22    82007
2019-05-21    79254
2019-05-16    77878
2019-05-08    76389
2019-05-24    74857
2019-05-20    74489
2019-05-15    74307
2019-05-18    73143
2019-05-07    72824
2019-05-17    72112
2019-05-19    69920
2019-05-06    69914
2019-05-11    69788
2019-05-02    68701
2019-05-10    68039
2019-05-09    66681
2019-05-27    66595
2019-05-03    62405
2019-05-01    60897
2019-05-25    59492
2019-05-28    59432
2019-05-23    56575
2019-05-04    56023
2019-05-29    55089
2019-05-30    52481
2019-05-26    49950
2019-05-14    42780
2019-05-13    22296
2019-05-05    16169
2019-05-12    10729
Name: starttime, dtype: int64